<a href="https://colab.research.google.com/github/PeymanKazemi/NLP_course2021_2/blob/main/code/Fine_tune_Sick_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     |████████████████████████████████| 306 kB 6.8 MB/s 
     |████████████████████████████████| 133 kB 70.4 MB/s 
     |████████████████████████████████| 67 kB 7.4 MB/s 
     |████████████████████████████████| 243 kB 70.0 MB/s 
     |████████████████████████████████| 1.1 MB 59.6 MB/s 
     |████████████████████████████████| 144 kB 72.4 MB/s 
     |████████████████████████████████| 271 kB 68.5 MB/s 
     |████████████████████████████████| 160 kB 75.8 MB/s 


In [2]:
!pip install -U transformers

     |████████████████████████████████| 3.4 MB 7.2 MB/s 
     |████████████████████████████████| 596 kB 71.7 MB/s 
     |████████████████████████████████| 895 kB 69.6 MB/s 
     |████████████████████████████████| 3.3 MB 63.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from datasets import load_dataset
raw_dataset = load_dataset('sick')

Downloading:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/218k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset sick downloaded and prepared to /root/.cache/huggingface/datasets/sick/default/0.0.0/c6b3b0b44eb84b134851396d6d464e5cb8f026960519d640e087fe33472626db. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'],
        num_rows: 4439
    })
    validation: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'],
        num_rows: 495
    })
    test: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'],
        num_rows: 4906
    })
})

In [5]:
new_dataset = raw_dataset.remove_columns([ 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'])
new_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA'],
        num_rows: 4439
    })
    validation: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA'],
        num_rows: 495
    })
    test: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA'],
        num_rows: 4906
    })
})

In [6]:
def sum_of_AB(example):
  example['entailment_BA'] =  example['sentence_A'] + "'" + ', '+   "'" + example['sentence_B']
  return example

updated_dataset = new_dataset.map(sum_of_AB)
updated_dataset['train']['entailment_BA'][0]

  0%|          | 0/4439 [00:00<?, ?ex/s]

  0%|          | 0/495 [00:00<?, ?ex/s]

  0%|          | 0/4906 [00:00<?, ?ex/s]

"A group of kids is playing in a yard and an old man is standing in the background', 'A group of boys in a yard is playing and a man is standing in the background"

In [7]:
# dataset = dataset.rename_column("sentence1", "sentenceA")

updated_dataset = updated_dataset.rename_column('entailment_BA','sentences')

In [8]:
updated_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'sentences'],
        num_rows: 4439
    })
    validation: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'sentences'],
        num_rows: 495
    })
    test: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'sentences'],
        num_rows: 4906
    })
})

In [9]:
finilized_dataset_01 = updated_dataset.remove_columns([ 'id', 'sentence_A', 'sentence_B','relatedness_score', 'entailment_AB'])
finilized_dataset_01

DatasetDict({
    train: Dataset({
        features: ['label', 'sentences'],
        num_rows: 4439
    })
    validation: Dataset({
        features: ['label', 'sentences'],
        num_rows: 495
    })
    test: Dataset({
        features: ['label', 'sentences'],
        num_rows: 4906
    })
})

Tokenizing

In [10]:
from transformers import AutoTokenizer
#checkpoint = "sentence-transformers/all-distilroberta-v1"
checkpoint = "bert-base-cased"


tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# tokenizer = AutoTokenizer.from_pretrained("eugenesiow/bart-paraphrase")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [11]:
def tokenizer_function(example):
  return tokenizer(example['sentences'], padding="max_length", truncation=True)


In [12]:
tokenized_datasets = finilized_dataset_01.map(tokenizer_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [13]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000)) 
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [14]:
small_train_dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'sentences', 'token_type_ids'],
    num_rows: 1000
})

- TF

In [16]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
tf_train_dataset = small_train_dataset.remove_columns(["sentences"]).with_format("tensorflow")
tf_eval_dataset = small_eval_dataset.remove_columns(["sentences"]).with_format("tensorflow")

In [19]:
train_features = {x: tf_train_dataset[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

eval_features = {x: tf_eval_dataset[x] for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["label"]))
eval_tf_dataset = eval_tf_dataset.batch(8)

In [20]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=3)

Epoch 1/3
125/125 [==============================] - 156s 1s/step - loss: nan - sparse_categorical_accuracy: 0.3010 - val_loss: nan - val_sparse_categorical_accuracy: 0.3030
Epoch 2/3
125/125 [==============================] - 138s 1s/step - loss: nan - sparse_categorical_accuracy: 0.2950 - val_loss: nan - val_sparse_categorical_accuracy: 0.3030
Epoch 3/3
125/125 [==============================] - 139s 1s/step - loss: nan - sparse_categorical_accuracy: 0.2950 - val_loss: nan - val_sparse_categorical_accuracy: 0.3030


In [ ]:
import tensorflow as tf
#from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F



# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained(checkpoint)

#model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

In [ ]:
#from transformers import AutoModelForSequenceClassification
#model = AutoModelForSequenceClassification.from_pretrained("eugenesiow/bart-paraphrase", num_labels=2)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

In [ ]:
trainer.train()

In [ ]:
import pandas as pd

df_train = pd.DataFrame(data = raw_dataset['train'])
df_test = pd.DataFrame(data = raw_dataset['test'])
df_validation = pd.DataFrame(data = raw_dataset['validation'])

In [ ]:
# merge dataframes
df_general = pd.concat([df_train, df_test, df_validation])
df_general.head()

In [ ]:
refined_df = df_general[['sentence_A', 'sentence_B','label', 'relatedness_score', 'entailment_AB']]
refined_df.head()

In [ ]:
import math
def sigmoid(x): 
    return 1 / (1 + math.exp(-x))


refined_df['label_new'] = refined_df.relatedness_score.apply(sigmoid)
refined_df

In [ ]:
refined_df.reset_index(drop=True, inplace=True)
refined_df

In [ ]:
dataset = refined_df[['sentence_A', 'sentence_B', 'label_new']]
dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("eugenesiow/bart-paraphrase")

# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [ ]:
item_sentence  = [["Hello, I'm a single sentence! Thist great.","I linke it. At the most."],
                          ["test bacht2. test", "test2. test"]]


In [ ]:
inputs = tokenizer(item_sentence, padding="max_length", truncation=True)
inputs

In [ ]:
# return one column

In [ ]:
def tokenizer_function(example):
  return tokenizer(example, padding="max_length", truncation=True)



In [ ]:
dataset.astype({'sentence_A': 'string', 'sentence_B': 'string', 'label_new': 'float64' }).dtypes

In [ ]:
import numpy as np

In [ ]:
test = dataset['label_new']
type(test)

In [ ]:
df_A = dataset['sentence_A'].apply(tokenizer_function)
df_B = dataset['sentence_B'].apply(tokenizer_function)
df_label = dataset['label_new']
df_tokenized =  pd.concat([df_A, df_B, df_label], axis=1)

In [ ]:
df_tokenized.head()

In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

tfds_token = datasets.Dataset(df_tokenized)

In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

df = df_tokenized
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())



In [ ]:
### convert to Huggingface dataset
hg_dataset = Dataset(pa.Table.from_pandas(df))

In [ ]:
df_tokenized.to_csv(index=False)

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/df_sick_tokenized.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_tokenized.to_csv(f)

In [ ]:
from datasets import Dataset
import pandas as pd

In [ ]:
dataset = Dataset.from_pandas(df_tokenized)

In [ ]:
type(df_tokenized['sentence_A'])

In [ ]:
train_labels = df_tokenized['label_new'].to_numpy().flatten()
train_labels

In [ ]:
features = df_tokenized.columns
features = list(features)
features

In [ ]:
#features = np.asarray(features).astype(np.int)
features = np.asarray(features).astype(np.string_)
#X_train=np.asarray(X_train).astype(np.int)
#y_train=np.asarray(y_train).astype(np.int)

In [ ]:
#TF dataset
dataset = tf.data.Dataset.from_tensor_slices((df_tokenized, features))


In [ ]:
for features_tensor, target_tensor in training_dataset:
    print(f'features:{features_tensor} target:{target_tensor}')

In [ ]:
'''
features = ['feature1', 'feature2', 'feature3']
print(training_df)

training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(training_df[features].values, tf.float32),
            tf.cast(training_df['target'].values, tf.int32)
        )
    )
)

for features_tensor, target_tensor in training_dataset:
    print(f'features:{features_tensor} target:{target_tensor}')
  '''


In [ ]:
import pandas as pd
import tensorflow as tf


Features,labels = df_tokenized,df_tokenized.pop('label_new')

dataset = tf.data.Dataset.from_tensor_slices((dict(Features), labels))
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
  sess.run(iterator.initializer)
  print(sess.run(next_element))
  print(sess.run(next_element))

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000)) 
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

In [ ]:
trainer.train()

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)